# Cluster GCN on Neo4j

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

This example demonstrates how to run Cluster GCN on a dataset stored entirely on disk with Neo4j. Our Neo4j Cluster GCN implementation iterates through user specified graph clusters and only ever stores the edges and features of one cluster in memory at any given time. This enables Cluster GCN to be used on extremely large datasets that don't fit into memory. 


We use Cora here as an example, see [this notebook](./load-cora-into-neo4j.ipynb) for instructions on how to load the Cora dataset into Neo4j. 

In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.1.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0b")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import stellargraph as sg
from stellargraph.connector.neo4j import Neo4jClusterNodeGenerator, Neo4jStellarGraph
from stellargraph.layer import ClusterGCN
import tensorflow as tf
import py2neo
import os
from sklearn import preprocessing, feature_extraction, model_selection

import numpy as np
import scipy.sparse as sps
import pandas as pd

## Connect to Neo4j

First we connect to the Neo4j with `py2neo`, we then create a `Neo4jStellarGraph` object.

In [4]:
default_host = os.environ.get("STELLARGRAPH_NEO4J_HOST")

# Create the Neo4j Graph database object;
# the arguments can be edited to specify location and authentication
graph = py2neo.Graph(host=default_host, port=None, user=None, password="pass")
neo4j_sg = Neo4jStellarGraph(graph)

/Users/kieranricardo/anaconda3/envs/tf2-stellar/lib/python3.6/site-packages/ipykernel_launcher.py:6: ExperimentalWarning: Neo4jStellarGraph is experimental: the class is not tested (see: https://github.com/stellargraph/stellargraph/issues/1578). It may be difficult to use and may have major changes at any time.
  


## Data labels

Here we get the node labels. Cluster GCN is semi-supervised and requires labels to be specified for some nodes.

In [5]:
# read the node labels from a seperate file
# note this function also returns a StellarGraph
# which we won't be using for this demo - we only need Neo4jStellarGraph!
_, labels = sg.datasets.Cora().load()

In [6]:
# split the node labels into train/test/val

train_labels, test_labels = model_selection.train_test_split(
    labels, train_size=140, test_size=None, stratify=labels
)
val_labels, test_labels = model_selection.train_test_split(
    test_labels, train_size=500, test_size=None, stratify=test_labels
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_labels)
val_targets = target_encoding.transform(val_labels)
test_targets = target_encoding.transform(test_labels)

## Neo4j Clustering

We use the Neo4j Data Science Library's community detection to split our graph into clusters for ClusterGCN.

In [8]:
clusters = neo4j_sg.cluster()

## Keras sequences

We now use `StellarGraph` to create keras sequences for taining, testing, and validation. Under the hood, these sequences connect to your Neo4j database and lazily load data for each cluster.

In [10]:
# create the Neo4jClusterNodeGenerator
# and the keras sequence objects

generator = Neo4jClusterNodeGenerator(neo4j_sg, clusters=clusters)

train_gen = generator.flow(train_labels.index, targets=train_targets)
val_gen = generator.flow(val_labels.index, targets=val_targets)
test_gen = generator.flow(test_labels.index, targets=test_targets)

Number of clusters 160
0 cluster has size 237
1 cluster has size 427
2 cluster has size 172
3 cluster has size 210
4 cluster has size 54
5 cluster has size 236
6 cluster has size 145
7 cluster has size 79
8 cluster has size 133
9 cluster has size 190
10 cluster has size 75
11 cluster has size 62
12 cluster has size 3
13 cluster has size 22
14 cluster has size 61
15 cluster has size 1
16 cluster has size 44
17 cluster has size 5
18 cluster has size 12
19 cluster has size 27
20 cluster has size 1
21 cluster has size 40
22 cluster has size 2
23 cluster has size 18
24 cluster has size 3
25 cluster has size 5
26 cluster has size 8
27 cluster has size 14
28 cluster has size 3
29 cluster has size 2
30 cluster has size 15
31 cluster has size 2
32 cluster has size 1
33 cluster has size 1
34 cluster has size 4
35 cluster has size 5
36 cluster has size 15
37 cluster has size 5
38 cluster has size 10
39 cluster has size 1
40 cluster has size 4
41 cluster has size 3
42 cluster has size 6
43 cluster

/Users/kieranricardo/anaconda3/envs/tf2-stellar/lib/python3.6/site-packages/ipykernel_launcher.py:4: ExperimentalWarning: Neo4jClusterNodeGenerator is experimental: the class is not tested (see: https://github.com/stellargraph/stellargraph/issues/1578). It may be difficult to use and may have major changes at any time.
  after removing the cwd from sys.path.


## Create and train your model!

Now we create and train the Cluster GCN model.

In [11]:
# create the model
cluster_gcn = ClusterGCN(
    layer_sizes=[32, 32], generator=generator, activations=["relu", "relu"], dropout=0.5,
)

x_in, x_out = cluster_gcn.in_out_tensors()
predictions = tf.keras.layers.Dense(units=val_targets.shape[1], activation="softmax")(
    x_out
)
model = tf.keras.Model(x_in, predictions)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.01),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=["acc"],
)

In [ ]:
# train the model!
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 160 steps, validate for 160 steps
Epoch 1/10
133/160 [=======================>......] - ETA: 0s - loss: 0.4840 - acc: 0.2178

In [ ]:
# evaluate the model
model.evaluate(test_gen)

In [ ]:
sg.utils.plot_history(history)

And that's it! We've trained a graph neural network without ever loading the whole graph into memory.

Please refer to [cluster-gcn-node-classification](./../../node-classification/cluster-gcn-node-classification.ipynb) for **node embedding visualization**.

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/connector/neo4j/cluster-gcn-on-cora-neo4j-example.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>